In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [2]:

# Carregue os DataFrames origem_ID e text do CSV


caminho_arquivo_1= 'cod_material_servico.txt'
origem_ID = pd.read_csv(caminho_arquivo_1,sep=";" )

caminho_arquivo_2= 'text.txt'
text = pd.read_csv(caminho_arquivo_2,sep="," )


In [ ]:

# Função para encontrar a correspondência fuzzy e retornar a correspondência com o maior score
def find_best_match(row):
    matches = process.extractOne(row['classif'], text['dsc_item'], scorer=fuzz.ratio)  # Substitua 'coluna_com_nome_real' pelo nome real da coluna em 'text'
    return pd.Series([row['classif'], matches[0]])

# Aplicar a função de correspondência fuzzy e criar o novo DataFrame
matches_df = origem_ID.apply(find_best_match, axis=1)
matches_df.columns = ['classif', 'melhor_classif']

# Salvar o DataFrame resultante em um novo CSV
matches_df.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(matches_df.head())

